### Eintragen der Entitäten ins Json

in diesem notebook werden die Entitäten annotiert.
1. Textzeilen in fortlaufenden String wandeln
2. Textstring Bildweise ausgeben lassen
3. Entitäten manuell setzen und speichern

---

#### 1. Zeilenweise gespeicherten ground truth in string umwandeln und speichern

In [1]:
import json

# JSON laden
with open('original_text.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Fließtext pro Eintrag erzeugen und in "text" speichern
for item in data:
    lines = item.get("lines", [])
    text = " ".join(line.strip() for line in lines)
    item["text"] = text

# Optional: Ergebnis speichern
with open('original_text.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

# Beispiel-Ausgabe zur Kontrolle
for item in data:
    print(f"{item['file_name']}:\n{item['text']}\n")


0001.jpg:
SPRACHCAFE WEIHNACHTSFEIER DIENSTAG 21.12. OLOF-PALME ZENTRUM 19 UHR WIR FREUEN UNS AUF EUCH!

0002.jpg:
SCHLIESST EUCH AN! ET Video & Content nichtohneuns_Dresden KUNDGEBUNG #SOLIDARITÄT MIT ALLEN! BERICHTE: WIE WAR DIE BERLIN-DEMO WIRKLICH? Dresden, Großer Garten, 8.8. um 16 Uhr

0003.jpg:
»ES IST GESCHEHEN UND FOLGLICH KANN ES WIEDER GESCHEHEN.« Primo Levi Gedenkkundgebung und antifaschistische Demonstration Mahnmal Levetzowstraße, Berlin-Moabit 9. November 1938–2023 18:00 Uhr 85 Jahre nach den Novemberpogromen; Solidarität mit den Opfern des deutschen Antisemitismus und Rassismus! 9november.blackblogs.org

0004.jpg:
FEIERABEND SINGEN KULTUR NETZ Meiendorf IMMER UM 17.00 UHR JEDE*R IST ♥-LICH WILLKOMMEN! MITTWOCH, 21. JUNI 2023 Wo: Grundschule Islandstraße, Islandstraße 25

0006.jpg:
IHR HABT UNS DIE STADT GESTOHLEN GEBT SIE WIEDER HER! ZUKUNFT STATT OSTKREUZ CAMPUS DEMO GEGEN GENTRIFIZIERUNG 09. JULI 2022 / 15:00 UHR

0007.jpg:
9. Mai 2024 Demo Herrentag Tag der Befreiung

#### 2. Textstring pro Bild ausgeben lassen

In [24]:
# text des eintrages/fotos ausgeben lassen

import json
#with open("../data/original_text.json", "r", encoding="utf-8") as f:
#    data = json.load(f)

with open("../data/data_annotated.json", "r", encoding="utf-8") as f:
    data = json.load(f)

#eintrag=data[220]
eintrag = next(e for e in data if e["file_name"] == "0207.jpg")

print(eintrag["text"])

#for ent in eintrag["entities"]:
#   print(ent["label"], ent["text"])


Frankfurt bleibt bunt! Schöner ohne AfD! 16. August - ab 15:30 Uhr Nähe Marktplatz Gegenkundgebung mit Kultur


#### 3. Entitäten manuell setzen und speichern

In [25]:
# label manuell setzen und speichern

import json

# -----------------------------
# 1. JSON laden
# -----------------------------
#with open("original_text.json", "r", encoding="utf-8") as f:
#   data = json.load(f)

with open("../data/data_annotated.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# -----------------------------
# 2. Entity-Finder-Funktion
# -----------------------------
def find_multiple_phrase_occurrences(text, entities_input):
    spans = []
    used_positions = []

    for phrase, label in entities_input:
        start = 0
        found = False
        while True:
            idx = text.find(phrase, start)
            if idx == -1:
                break
            end = idx + len(phrase)

            # Falls Position schon verwendet → überspringen
            if any(idx >= u[0] and idx < u[1] for u in used_positions):
                start = end
                continue

            spans.append({
                "start": idx,
                "end": end,
                "label": label,
                "text": text[idx:end]
            })
            used_positions.append((idx, end))
            found = True
            break  # nur ein Match pro entity_input-Zeile
        if not found:
            print(f"Phrase '{phrase}' nicht gefunden oder alle Vorkommen bereits belegt.")
    return spans



# -----------------------------
# 3. Annotation für Eintrag xy
# -----------------------------
# Phrasen + Label eingeben

# ("13.08.2022 — 15.08.2022", "DATE"),
# ("16:00", "TIME"),
# ("Kundgebung", "EVENT"),
# ("DAS GROSSE FINALE", "TOPIC"),
# ("Marktplatz Ried", "LOC"),
# ("LESECLUB MEIENDORF", "ORG"),


entities_input = [
    
    ("Gegenkundgebung", "EVENT"),
    ("Offenes Antifa Treffen", "EVENT"),
    ("Abendessen", "EVENT"),
    ("DISKUSSION", "EVENT"),
   
    ("Frankfurt bleibt bunt!", "TOPIC"),
    ("Schöner ohne AfD!", "TOPIC"),
    ("gegen die European Gas Conference", "TOPIC"),
    ("POWER TO THE PEOPLE", "TOPIC"),
    ("Zeigt Eure Künste!", "TOPIC"),
    ("Erhebt Eure Stimme!", "TOPIC"),
    
    
    ("16. August", "DATE"),
    ("31.12.2024", "DATE"),
    ("16.07.24", "DATE"),
    
    ("15:30 Uhr", "TIME"),
    ("20 Uhr", "TIME"),
    ("22:30 Uhr", "TIME"),

    
    ("Marktplatz", "LOC"),
    ("Lottumstr. 10a", "LOC"),
    ("Prenzlauer Berg", "LOC"),
 

    
]

# Text holen
#entry = data[221]
entry = next(e for e in data if e["file_name"] == "0207.jpg")
text = entry["text"]

# Automatisch Entitäten zuordnen
entry["entities"] = find_multiple_phrase_occurrences(text, entities_input)

# -----------------------------
# 4. JSON speichern
# -----------------------------
with open("../data/data_annotated.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4) # ascii false, damit umlaute bleiben, indent 4, einrückung um vier zeichen, für mehr übersichtlichkeit

# -----------------------------
# 5. Kontrolle
# -----------------------------
print("Beispielausgabe:")
for e in entry["entities"]:
    print(f"{e['label']}: {text[e['start']:e['end']]} ({e['start']}, {e['end']})")


Phrase 'Offenes Antifa Treffen' nicht gefunden oder alle Vorkommen bereits belegt.
Phrase 'Abendessen' nicht gefunden oder alle Vorkommen bereits belegt.
Phrase 'DISKUSSION' nicht gefunden oder alle Vorkommen bereits belegt.
Phrase 'gegen die European Gas Conference' nicht gefunden oder alle Vorkommen bereits belegt.
Phrase 'POWER TO THE PEOPLE' nicht gefunden oder alle Vorkommen bereits belegt.
Phrase 'Zeigt Eure Künste!' nicht gefunden oder alle Vorkommen bereits belegt.
Phrase 'Erhebt Eure Stimme!' nicht gefunden oder alle Vorkommen bereits belegt.
Phrase '31.12.2024' nicht gefunden oder alle Vorkommen bereits belegt.
Phrase '16.07.24' nicht gefunden oder alle Vorkommen bereits belegt.
Phrase '20 Uhr' nicht gefunden oder alle Vorkommen bereits belegt.
Phrase '22:30 Uhr' nicht gefunden oder alle Vorkommen bereits belegt.
Phrase 'Lottumstr. 10a' nicht gefunden oder alle Vorkommen bereits belegt.
Phrase 'Prenzlauer Berg' nicht gefunden oder alle Vorkommen bereits belegt.
Beispielausgab